In [1]:
import json
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Đọc dữ liệu từ tập tin JSON
file_path = "hsg.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Tạo DataFrame
df = pd.DataFrame(
    {
        "Date": pd.to_datetime(data["t"], unit="s"),
        "Open": data["o"],
        "High": data["h"],
        "Low": data["l"],
        "Close": data["c"],
        "Volume": data["v"],
    }
)

# Sử dụng giá đóng cửa (Close) của ngày trước để dự đoán giá mở cửa (Open) ngày tiếp theo
df["Next Open"] = df["Open"].shift(-1)
df = df.dropna()  # Xóa bỏ hàng cuối cùng vì không có giá mở cửa cho "ngày mai"

# Chia dữ liệu thành features (X) và target (y)
X = df[["High", "Low", "Close", "Volume"]]
y = df["Next Open"]

# Chuẩn hóa dữ liệu
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Chia dữ liệu để lấy dữ liệu mới nhất làm input cho việc dự đoán
X_train, X_predict = X_scaled[:-1], X_scaled[-1:]
y_train = y_scaled[:-1]

# Xây dựng mô hình GRU
model = Sequential(
    [
        GRU(
            50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])
        ),
        GRU(50),
        Dense(1),
    ]
)

# Biên dịch mô hình
model.compile(optimizer="adam", loss="mean_squared_error")

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=50, batch_size=1024, verbose=1)

# Dự đoán giá mở cửa cho ngày mai
predicted_next_open_scaled = model.predict(X_predict)

# Sử dụng inverse_transform để chuyển dự đoán về cùng quy mô với dữ liệu gốc
predicted_next_open = scaler_y.inverse_transform(predicted_next_open_scaled)

print(f"Giá mở cửa dự đoán cho ngày mai: {predicted_next_open[0][0]}")

2024-03-25 20:38:45.287571: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-25 20:38:45.287601: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-25 20:38:45.287609: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-25 20:38:45.287772: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-25 20:38:45.287795: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-03-25 20:38:47.534724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 8s 49ms/step - loss: 0.1234
Epoch 2/100
120/120 [==============================] - 2s 20ms/step - loss: 0.0041
Epoch 3/100
120/120 [==============================] - 2s 14ms/step - loss: 0.0030
Epoch 4/100
120/120 [==============================] - 2s 18ms/step - loss: 0.0022
Epoch 5/100
120/120 [==============================] - 2s 15ms/step - loss: 0.0016
Epoch 6/100
120/120 [==============================] - 2s 15ms/step - loss: 0.0012
Epoch 7/100
120/120 [==============================] - 2s 13ms/step - loss: 9.6736e-04
Epoch 8/100
120/120 [==============================] - 2s 13ms/step - loss: 8.4749e-04
Epoch 9/100
120/120 [==============================] - 1s 12ms/step - loss: 8.8009e-04
Epoch 10/100
120/120 [==============================] - 2s 13ms/step - loss: 8.3825e-04
Epoch 11/100
120/120 [==============================] - 2s 14ms/step - loss: 8.3172e-04
Epoch 12/100
120/120 [==============================] - 2s 14ms/step - loss: 